## Final Version

## Here we will use the astropy as our fits reader

In [ ]:
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
import os

rcParams['figure.figsize'] = [10., 8.]

## Here we will open our FITS file

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
hdu_list.info()

## Gather information from the PRIMARY block using hdu_list

In [ ]:
image_data = hdu_list[0].data

## To gather dimensions of the image we will look at the shape of the array

In [ ]:
print(type(image_data))
print(image_data.shape)

## Close the FITS file, everything was stored to a variable

In [ ]:
hdu_list.close()

## shortcut: [getdata()] to just read in the image data and close the file

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

## Show the image

In [ ]:
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

plt.savefig('astro_1.png')

## Subtract the background data before sources can be detected with separate steps

In [ ]:
#measure a spatially varying background on the image
image_data = image_data.byteswap(False).newbyteorder()
bkg = sep.Background(image_data)

## Basic statistics about our image:

In [ ]:
print('Min:', np.min(image_data))
print('Max:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('Stdev:', np.std(image_data))

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(image_data, cmap='cool', norm=LogNorm())


cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4])
cbar.ax.set_yticklabels(['5,000','10,000','20,000'])

plt.savefig('image.png')

## Control the box size

In [ ]:
bkg = sep.Background(image_data)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background of the image
plt.imshow(bkg_image, interpolation='nearest', cmap='magma', origin='lower')
plt.colorbar();

plt.savefig('backgroun_noise.png')

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='rainbow', origin='lower')
plt.colorbar();

plt.savefig('background_noise_2.png')

In [ ]:
# subtract the background
data_sub = image_data - bkg

## Now that we’ve subtracted the background, we can run object detection on the background-subtracted data. You can see the background noise level is pretty flat. So here we’re setting the detection threshold to be a constant value of 1.5σ where σ is the global background RMS.

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

#sep.extract has many options for controlling detection threshold, pixel masking, filtering, and object deblending.

## Number of sources detected

In [ ]:
# how many objects were detected
len(objects)

## objects['x'] and objects['y'] will give the centroid coordinates of the objects. Just to check where the detected objects are, we’ll over-plot the object coordinates with some basic shape parameters on the image

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='plasma',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('black')
    ax.add_artist(e)
    
plt.savefig('ellipse.png')

## object has many fields, gives information such as second moments, peak pixel position and values

In [ ]:
# available fields
objects.dtype.names

## We’ll perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, 
                                     objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)


In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

## Using ABMag we plot the Histogram (referenced from STSCI website)

In [ ]:
#we are only concerned with positive flux values
flux_1 = flux[flux > 0]
ABMag = -2.5 * np.log10(flux_1) - 26.0974

In [ ]:
histogram = plt.hist(ABMag, bins = 'auto')
plt.savefig('astro_histogram.png')

In [ ]:
print('min', min(flux))
print('max', max(flux))
print('shape', flux.shape)